In [ ]:
from db_inference import simple_neo4j

In [ ]:
db = simple_neo4j.Neo4jConnection()

BAIT_HASH = "7ba89c248cf6a20732"

In [ ]:
%%time
query = (
    'MATCH (bait:P30)-->(bn:P90)-->(bp:P100)-[e]-(tp:P100)<--(tn:P90)<--(tgt:P30) '
    f'WHERE bait.p30 = "{BAIT_HASH}" '
    'WITH tgt, count(e) AS numShared '
    'MATCH (tgt:P30) --> (tgt_n:P90) --> (tgt_p:P100) '
    'WITH tgt, numShared, count(tgt_p) as numProts '
    'RETURN tgt{.p30, numShared, numProts}'
)

resp = db.query(query)
{r[0]['p30']: (r[0]['numShared'], r[0]['numProts']) for r in resp}

In [ ]:
{r[0]['p30']: (r[0]['numShared'], r[0]['numProts']) for r in resp}

In [ ]:
query = (
    'MATCH (bait:P30)-->(bn:P90)-->(bp:P100)-[e]-(tp:P100)<--(tn:P90)<--(tgt:P30) '         
    f'WHERE bait.p30 = "{BAIT_HASH}" '
    'WITH tgt, count(e) AS numShared '
    'RETURN tgt{.p30, numShared} '
)

resp = db.query(query)
{r[0]['p30']: r[0]['numShared'] for r in resp}



In [ ]:
query = 'MATCH (bait:P30)-->(bn:P90)-->(bp:P100)--(tp:P100)<--(tn:P90)<--(tgt:P30) ' \
        f'WHERE bait.p30 = "7ba89c248cf6a20732" RETURN DISTINCT collect(tgt.p30)'
resp = db.query(query)
return resp[0][0]
# tgt_nodes = resp[0][0]
# tgts = [tgt["p30"] for tgt in tgt_nodes if tgt["p30"] != p30_hash]
# return tgts

# tgts = get_targets_for_bait(BAIT_HASH)
# print(len(tgts))
resp[0][0]

In [ ]:
def get_targets_for_bait(p30_hash):
    query = 'MATCH (bait:P30)-->(bn:P90)-->(bp:P100)--(tp:P100)<--(tn:P90)<--(tgt:P30) ' \
            f'WHERE bait.p30 = "{p30_hash}" RETURN DISTINCT collect(tgt.p30)'
    resp = db.query(query)
    tgts = resp[0][0]
    return tgts

tgts = get_targets_for_bait(BAIT_HASH)
print(len(tgts))

In [ ]:
def get_num_p100s(p30_hash): 
    query = f'MATCH (n:P30)-->(:P90)-->(p:P100) WHERE n.p30 = "{p30_hash}" RETURN count(p)'
    resp = db.query(query)
    num_p100s = resp[0]['count(p)']
    return num_p100s

get_num_p100s(BAIT_HASH)

In [ ]:
for tgt in tgts[0:5]:
    print(tgt, get_num_p100s(tgt))

In [ ]:
def get_num_shared(p30_A, p30_B):
    query = 'MATCH (n:P30)-->(o:P90)-->(p:P100)-[e]-(tp:P100)<--(to:P90)<--(tn:P30) ' \
            f'WHERE n.p30 = "{p30_A}" AND tn.p30="{p30_B}" RETURN count(e), count(p), count(tp)'
    resp = db.query(query)
    count_edges = resp[0]["count(e)"]
    count_prot_A = resp[0]["count(p)"]
    count_prot_B = resp[0]["count(tp)"]
    assert count_edges == count_prot_A == count_prot_B
    return count_edges
    
get_num_shared(p30_A = "32203c2ec32f8968bb", p30_B = "7ba89c248cf6a20732")

In [ ]:
%%time
bait_num_p100s = get_num_p100s(BAIT_HASH)
for tgt in tgts:
    tgt_num_p100s = get_num_p100s(tgt)
    num_shared = get_num_shared(BAIT_HASH, tgt)
    
    print(f"icity {tgt} -> {BAIT_HASH} = {num_shared / tgt_num_p100s:.4f}  ({num_shared:4d} / {tgt_num_p100s:4d})")
    print(f"icity {BAIT_HASH} -> {tgt} = {num_shared / bait_num_p100s:.4f}  ({num_shared:4d} / {bait_num_p100s:4d})")

In [ ]:
# def get_p30_for_protein(p100_hash):
p100_hash = "e22cef4bf838680478"
query = f'MATCH (n:P30)-->(o:P90)-->(p:P100) WHERE p.p100 = "{p100_hash}" RETURN n'
resp = db.query(query)
p30_hash = resp[0][0]['p30']